# Time Dependencies

In [1]:
# Following code is needed to preconfigure this notebook
import datetime
import sys
import os
sys.path.insert(0, os.path.abspath('../../..'))

import pyflow as pf

Sometimes you want a task to run at a given time, or to run every three hours, or to run only on the first of the month, or on Mondays. For that, **pyflow** supports date and time dependencies.

Like triggers, date and time dependencies can be set for a family. In this case, the tasks of this family will only run according to these dependencies.

In [2]:
with pf.Suite('test') as s:
    with pf.Family('f2'):
        pf.Variable('SLEEP', 20)
        t1 = pf.Task('t1')
        t2 = pf.Task('t2')
        t3 = pf.Task('t3')
        t4 = pf.Task('t4')
        t5 = pf.Task('t5')

    t1.time = '00:30 23:30 00:30'  # start(hh:mm) end(hh:mm) increment(hh:mm)
    t2.day = 'thursday'            # thursday at 1 pm
    t2.time = '13:00'
    t3.time = '0 12 * * *'         # time is not considered until date is free
    t3.date = '1.*.*'              # `day.month.year` - * means every day, month, year
    t4.time = '+00:02'             # + means realative to suite begin/requeue time
    t5.time = '00:02'              # 2 minutes past midnight

s

Suite(/test)

## Time

Time dependencies can be absolute, i.e. they will run at the exact time. They can also be relative; in this case, we provide the time from the moment the suite is begun.

Time dependencies can be repeated at regular intervals. The nodes stay complete once all-time instances have run.

In [3]:
with pf.Suite('test') as s:
    with pf.Family('f3'):
        with pf.Task('t1') as t1:
            t1.time = '23:00'               # at next 23:00
        with pf.Task('t2') as t2:
            t2.time = '0 10-20 * * *'       # every hour between 10 am and 8 pm
        with pf.Task('t3') as t3:
            t3.time = '+00:01'              # one minute after the suite has begun
        with pf.Task('t4') as t4:
            t4.time = '+00:10 01:00 00:05'  # 10 to 60 minutes after begin every 5 minutes

s

Suite(/test)

In the last example, we have a task that runs every hour. However, if the task takes longer than an hour, the time slot will be missed.

## Cron

Cron dependencies can be specified using the `pf.Cron` class. Cron differs from time as when the node is complete it queues again immediately. Cron also only works with a real time clock, not a hybrid clock.

In [4]:
with pf.Suite('test') as s:
    with pf.Family('f4'):
        with pf.Task('t1') as t1:
            t1.cron = '0 23 * * *'                                           # every day at 11 pm
        with pf.Task('t2') as t2:
            t2.cron = '0 8-12 * * *'                                         # every hour between 8 and 12 am
        with pf.Task('t3') as t3:
            t3.cron = '0 11 * * SUN,TUE'                                     # every Sunday and Tuesday at 11 am
        with pf.Task('t4') as t4:
            t4.cron = '0 2 1,15 * *'                                         # every 1st and 15th of each month at 2 am
        with pf.Task('t5') as t5:
            t5.cron = '0 14 1 1 *'                                           # every first of January at 2 pm
        with pf.Task('t6'):
            pf.Cron('23:00', last_week_days_of_the_month=[5])                # every *last* Friday of month at 11 pm
        with pf.Task('t7'):
            pf.Cron('23:00', days_of_month=[1], last_day_of_the_month=True)  # every first and last of month at 11 pm

s

Suite(/test)

The **ecFlow** server records all commands sent to it, in a log file `<host>.<port>.ecf.log`. This log file will grow over time to a considerable size.

In the following example, we will create a task whose job is to periodically back up and clear this log file. This will be done with cron attribute. A cron will run indefinitely, i.e. once it has completed it will automatically re-queue itself.

In [5]:
with pf.Suite('test', host=pf.LocalHost(), files='/test') as s:
    with pf.Family('house_keeping'):
        with pf.Task('clear_log', script=[
            '# copy the log file to the ECF_HOME/log directory',
            'cp %ECF_LOG% %ECF_HOME%/log/.',
            '',
            '# clear the log file',
            'ecflow_client --log=clear',
        ]):
            pf.Cron('30 22 * * SUN') # run every Sunday at 10:30 pm

s

Suite(/test)

In [6]:
s.deploy_suite(pf.Notebook)

## Date or Day

Date dependencies can be specified using the `date` or `day` attribute. Date dependencies are always absolute, but wildcards can be used.

In [7]:
with pf.Suite('test') as s:
    with pf.Family('f5'):
        with pf.Task('t1') as t1:
            t1.date = '31.12.2012' # the 31st of December 2012
        with pf.Task('t2') as t2:
            t2.date = '01.*.*'     # every first of the month
        with pf.Task('t3') as t3:
            t3.date = '*.10.*'     # every day in October
        with pf.Task('t4') as t4:
            t4.date = '1.*.2008'   # every first of the month, but only in 2008
        with pf.Task('t5') as t5:
            t5.day = 'monday'      # every monday

s

Suite(/test)

## Mixing time dependencies on the same node

A single task can have many time and date dependencies.

In [8]:
with pf.Suite('test') as s:
    with pf.Family('f6'):
        with pf.Task('t1') as t1:
            t1.time = '10:00'                # here day acts like a guard over the time. i.e. time is not considered until Monday
            t1.day = 'monday'                # run on Monday at 10 am
        with pf.Task('t2') as t2:
            t2.time = '0 1,16 * * *'         # on the same node, day/date acts like a guard over the time attributes
            t2.date = ['01.*.*', '10.*.*']   # the first and tenth of every month and year
            t2.day = ['sunday', 'wednesday'] # the time is only set free *if* we are on one of the day/dates

s

Suite(/test)

The second task will run on Sundays and Wednesdays at 1 am and 4 pm, but only if the day is the 1st or the 10th of the month.

## Mixing time dependencies on different nodes

When time dependencies are placed on different nodes in the hierarchy, the results may seem surprising.

In [9]:
with pf.Suite('test') as s:
    with pf.Family('f1') as f1:
        f1.day = 'monday'     # the day *still* guards the time attribute
        with pf.Task('t1') as t1:
            t1.time = '10:00' # will run on Monday at 10 am
    with pf.Family('f2') as f2:
        f2.time = '10:00'
        with pf.Task('t2') as t2:
            t2.day = 'monday' # this will run on Monday morning at 00:00 and Monday at 10 am

s

Suite(/test)

The example above assumes we have a suite, with an infinite repeat loop.

The task `t2` will run on Monday morning at 00:00 because time dependencies on different nodes act independently of each other. In this case, the time attribute was set free on Sunday at 10 am (and once free it stays free until it is re-queued). Hence task `t2` is free to run on Monday morning. After the task has run and re-queued, it will then run on Monday at 10 am.

## Time Triggers

In [10]:
with pf.Suite('test') as s:
    with pf.Family('f1') as f1:
        with pf.Task('t1') as t1:
            t1.time = '23:00' # t1.triggers = ':TIME == 2300'
        with pf.Task('t2') as t2:
            t2.date = '1.*.*' # t2.triggers = ':DD == 1'
        with pf.Task('t3') as t3:
            t3.day = 'monday' # t3.triggers = ':DOW == 1'

s

Suite(/test)

The `:` means a search for the variable up the node tree.

Triggers can also use AND/OR logic and the full range of operators `<`, `>`, `<=`, `>=`.

In [11]:
with pf.Suite('test') as s:
    with pf.Family('f1') as f1:
        with pf.Task('t1') as t1:
            t1.time = '23:00'                           # time attributes
            t1.day = 'monday'
        with pf.Task('t2') as t2:
            t2.triggers = ':DOW == 1 and :TIME >= 1300' # time based trigger
        with pf.Task('t3') as t3:
            t3.triggers = ':TIME >= 1300'               # combination
            t3.day = 'monday'

s

Suite(/test)

In [12]:
with pf.Suite('test') as s:
    with pf.Family('f1') as f1:
        with pf.Task('t1') as t1:
            t1.triggers = ':ECF_DATE == 20200720 and :TIME >= 1000'
        with pf.Task('t2') as t2:
            t2.triggers = ':DOW == 4 and :TIME >= 1300'
        with pf.Task('t3') as t3:
            t3.triggers = ':DD == 1 and :TIME >= 1200'
        with pf.Task('t4') as t4:
            t4.triggers = '(:DOW == 1 and :TIME >= 1300) or (:DOW == 5 and :TIME >= 1000)'
        with pf.Task('t5') as t5:
            t5.triggers = ':TIME == 0002' # 2 minutes past midnight

s            

Suite(/test)